In [1464]:
import importlib
import types


def walk_reload(module: types.ModuleType) -> None:
    if hasattr(module, "__all__"):
        for submodule_name in module.__all__:
            walk_reload(getattr(module, submodule_name))
    importlib.reload(module)

import io
from utils import XML
#import importlib
walk_reload(XML)

import numpy as np
from urllib.request import urlopen
from datetime import datetime
from dateutil.relativedelta import *



In [1465]:
dep = XML.DepositInstrument(datetime(2018,1,10), datetime(2018,4,10), 2/100, USD_ratecurve)
dep.getZCsensi()


2018-02-12    0.000000
2018-03-12    0.000000
2018-04-10   -0.247743
2018-07-10    0.000000
2019-01-10    0.000000
2020-01-10    0.000000
2021-01-10    0.000000
2022-01-10    0.000000
2023-01-10    0.000000
2024-01-10    0.000000
2025-01-10    0.000000
2026-01-10    0.000000
2027-01-10    0.000000
2028-01-10    0.000000
2030-01-10    0.000000
2033-01-10    0.000000
2038-01-10    0.000000
2043-01-10    0.000000
2048-01-10    0.000000
dtype: float64

In [1466]:
swap = XML.FixFloatSwapInstrument(datetime(2018,1,10), datetime(2018,1,10) + relativedelta(years=2), 2, 6, 3, 2/100, USD_ratecurve, USD_ratecurve)
swap.getZCsensi()


2018-02-12    0.000000
2018-03-12    0.000000
2018-04-10    0.000000
2018-07-10    0.004975
2019-01-10    0.017351
2020-01-10    1.942747
2021-01-10    0.000000
2022-01-10    0.000000
2023-01-10    0.000000
2024-01-10    0.000000
2025-01-10    0.000000
2026-01-10    0.000000
2027-01-10    0.000000
2028-01-10    0.000000
2030-01-10    0.000000
2033-01-10    0.000000
2038-01-10    0.000000
2043-01-10    0.000000
2048-01-10    0.000000
dtype: float64

In [1467]:
# get the OIS curves (already calibrated) from the markit website
curveURL = "http://www.markit.com/news/InterestRates_USD_20180105.zip"
USD_ratecurve = XML.RateCurve(curveURL)

In [1468]:
# deposits interpolation
size = USD_ratecurve.df.shape[0]
filter_deposits = USD_ratecurve.df['source']=='deposit'
filter_swaps = USD_ratecurve.df['source']=='swap'
ytms = USD_ratecurve.df[filter_deposits]['YTM']
parrates = USD_ratecurve.df[USD_ratecurve.df['source']=='deposit']['parrate']
discounts = 1/(1+ytms*parrates)
USD_ratecurve.df.loc[USD_ratecurve.df['source']=='deposit','df'] = discounts
USD_ratecurve.df.loc[USD_ratecurve.df['source']=='deposit','ZCrate'] = -ytms*np.log(discounts)


In [1469]:
USD_ratecurve.df

,source,tenor,parrate,YTM,ZCrate,df
maturity,,,,,,
2018-02-12,deposit,1M,0.015525,0.083333,0.000108,0.998708
2018-03-12,deposit,2M,0.016219,0.166667,0.000450,0.997304
2018-04-10,deposit,3M,0.017039,0.250000,0.001063,0.995758
2018-07-10,deposit,6M,0.018651,0.500000,0.004641,0.990761
2019-01-10,deposit,1Y,0.021495,1.000000,0.021267,0.978957
2020-01-10,swap,2Y,0.021470,2.000000,0.021470,0.000000
2021-01-10,swap,3Y,0.022405,3.000000,0.022405,0.000000
2022-01-10,swap,4Y,0.022930,4.000000,0.022930,0.000000
2023-01-10,swap,5Y,0.023225,5.000000,0.023225,0.000000


In [1480]:
spot = USD_ratecurve.spotdate

for index in USD_ratecurve.df.index:
    line = USD_ratecurve.df.loc[index]
    source = line['source']
    parrate = line['parrate']
    maturity = index
    if (source == 'deposit'):
        USD_ratecurve.df.loc[index, "instrumentList"] = XML.DepositInstrument(spot, maturity, parrate, USD_ratecurve)
    elif (source == 'swap'):
        ytm = line['YTM']
        USD_ratecurve.df.loc[index, "instrumentList"] = XML.FixFloatSwapInstrument(spot, maturity, ytm, 6, 3, parrate, USD_ratecurve, USD_ratecurve)

In [1481]:
jacobienMatrix = pd.DataFrame(0, index = USD_ratecurve.df.index, columns = USD_ratecurve.df.index)
        
for i in USD_ratecurve.df.index:
    print("computation done at : ", datetime.now(tz=None))
    jacobienMatrix.loc[i] = USD_ratecurve.df.loc[i,'instrumentList'].getZCsensi()    
    USD_ratecurve.df.loc[i,'MV']  = USD_ratecurve.df.loc[i,'instrumentList'].getMV().iloc[0]

print(USD_ratecurve.df['MV']*1000000)

inv_J = pd.DataFrame(np.linalg.pinv(jacobienMatrix.values), jacobienMatrix.columns, jacobienMatrix.index)
# inv_J.dot(jacobienMatrix) cette quantité est presque une matrice diagonale
epsilon = -inv_J.dot(USD_ratecurve.df['MV'])
USD_ratecurve.df['ZCrate'] += epsilon
USD_ratecurve.cleanCache()

computation done at :  2020-04-20 21:19:59.458205
computation done at :  2020-04-20 21:19:59.584280
computation done at :  2020-04-20 21:19:59.670564
computation done at :  2020-04-20 21:19:59.745279
computation done at :  2020-04-20 21:19:59.811943
computation done at :  2020-04-20 21:19:59.875439
computation done at :  2020-04-20 21:20:00.124877
computation done at :  2020-04-20 21:20:00.488951
computation done at :  2020-04-20 21:20:00.947793
computation done at :  2020-04-20 21:20:01.546004
computation done at :  2020-04-20 21:20:02.216917
computation done at :  2020-04-20 21:20:03.001379
computation done at :  2020-04-20 21:20:03.903095
computation done at :  2020-04-20 21:20:04.898134
computation done at :  2020-04-20 21:20:06.026385
computation done at :  2020-04-20 21:20:07.369024
computation done at :  2020-04-20 21:20:09.038980
computation done at :  2020-04-20 21:20:11.349814
computation done at :  2020-04-20 21:20:14.187032
maturity
2018-02-12     2827.732327
2018-03-12    

In [1482]:
spot = USD_ratecurve.spotdate

for index in USD_ratecurve.df.index:
    line = USD_ratecurve.df.loc[index]
    source = line['source']
    parrate = line['parrate']
    maturity = index
    if (source == 'deposit'):
        USD_ratecurve.df.loc[index, "instrumentList"] = XML.DepositInstrument(spot, maturity, parrate, USD_ratecurve)
    elif (source == 'swap'):
        ytm = line['YTM']
        USD_ratecurve.df.loc[index, "instrumentList"] = XML.FixFloatSwapInstrument(spot, maturity, ytm, 6, 3, parrate, USD_ratecurve, USD_ratecurve)
        jacobienMatrix = pd.DataFrame(0, index = USD_ratecurve.df.index, columns = USD_ratecurve.df.index)
        
for i in USD_ratecurve.df.index:
    print("computation done at : ", datetime.now(tz=None))
    jacobienMatrix.loc[i] = USD_ratecurve.df.loc[i,'instrumentList'].getZCsensi()    
    USD_ratecurve.df.loc[i,'MV']  = USD_ratecurve.df.loc[i,'instrumentList'].getMV().iloc[0]

print(USD_ratecurve.df['MV']*1000000)

inv_J = pd.DataFrame(np.linalg.pinv(jacobienMatrix.values), jacobienMatrix.columns, jacobienMatrix.index)
# inv_J.dot(jacobienMatrix) cette quantité est presque une matrice diagonale
epsilon = -inv_J.dot(USD_ratecurve.df['MV'])
USD_ratecurve.df['ZCrate'] += epsilon
USD_ratecurve.cleanCache()

computation done at :  2020-04-20 21:20:48.148156
computation done at :  2020-04-20 21:20:48.260969
computation done at :  2020-04-20 21:20:48.343549
computation done at :  2020-04-20 21:20:48.434538
computation done at :  2020-04-20 21:20:48.501257
computation done at :  2020-04-20 21:20:48.564019
computation done at :  2020-04-20 21:20:48.860983
computation done at :  2020-04-20 21:20:49.221952
computation done at :  2020-04-20 21:20:49.748700
computation done at :  2020-04-20 21:20:50.324564
computation done at :  2020-04-20 21:20:51.039853
computation done at :  2020-04-20 21:20:51.949751
computation done at :  2020-04-20 21:20:52.926864
computation done at :  2020-04-20 21:20:53.950316
computation done at :  2020-04-20 21:20:55.063810
computation done at :  2020-04-20 21:20:56.408659
computation done at :  2020-04-20 21:20:58.105246
computation done at :  2020-04-20 21:21:00.418740
computation done at :  2020-04-20 21:21:03.459499
maturity
2018-02-12      3.983017
2018-03-12     1

In [1483]:
spot = USD_ratecurve.spotdate

for index in USD_ratecurve.df.index:
    line = USD_ratecurve.df.loc[index]
    source = line['source']
    parrate = line['parrate']
    maturity = index
    if (source == 'deposit'):
        USD_ratecurve.df.loc[index, "instrumentList"] = XML.DepositInstrument(spot, maturity, parrate, USD_ratecurve)
    elif (source == 'swap'):
        ytm = line['YTM']
        USD_ratecurve.df.loc[index, "instrumentList"] = XML.FixFloatSwapInstrument(spot, maturity, ytm, 6, 3, parrate, USD_ratecurve, USD_ratecurve)
        jacobienMatrix = pd.DataFrame(0, index = USD_ratecurve.df.index, columns = USD_ratecurve.df.index)
        
for i in USD_ratecurve.df.index:
    print("computation done at : ", datetime.now(tz=None))
    jacobienMatrix.loc[i] = USD_ratecurve.df.loc[i,'instrumentList'].getZCsensi()    
    USD_ratecurve.df.loc[i,'MV']  = USD_ratecurve.df.loc[i,'instrumentList'].getMV().iloc[0]

print(USD_ratecurve.df['MV']*1000000)

inv_J = pd.DataFrame(np.linalg.pinv(jacobienMatrix.values), jacobienMatrix.columns, jacobienMatrix.index)
# inv_J.dot(jacobienMatrix) cette quantité est presque une matrice diagonale
epsilon = -inv_J.dot(USD_ratecurve.df['MV'])
USD_ratecurve.df['ZCrate'] += epsilon
USD_ratecurve.cleanCache()

computation done at :  2020-04-20 21:21:10.794737
computation done at :  2020-04-20 21:21:10.915796
computation done at :  2020-04-20 21:21:11.003796
computation done at :  2020-04-20 21:21:11.082048
computation done at :  2020-04-20 21:21:11.145467
computation done at :  2020-04-20 21:21:11.218550
computation done at :  2020-04-20 21:21:11.483385
computation done at :  2020-04-20 21:21:11.859241
computation done at :  2020-04-20 21:21:12.421730
computation done at :  2020-04-20 21:21:13.075572
computation done at :  2020-04-20 21:21:13.792913
computation done at :  2020-04-20 21:21:14.610230
computation done at :  2020-04-20 21:21:15.561469
computation done at :  2020-04-20 21:21:16.600972
computation done at :  2020-04-20 21:21:17.765683
computation done at :  2020-04-20 21:21:19.159983
computation done at :  2020-04-20 21:21:21.025911
computation done at :  2020-04-20 21:21:23.497553
computation done at :  2020-04-20 21:21:26.364095
maturity
2018-02-12    7.932274e-06
2018-03-12    

In [1484]:
spot = USD_ratecurve.spotdate

for index in USD_ratecurve.df.index:
    line = USD_ratecurve.df.loc[index]
    source = line['source']
    parrate = line['parrate']
    maturity = index
    if (source == 'deposit'):
        USD_ratecurve.df.loc[index, "instrumentList"] = XML.DepositInstrument(spot, maturity, parrate, USD_ratecurve)
    elif (source == 'swap'):
        ytm = line['YTM']
        USD_ratecurve.df.loc[index, "instrumentList"] = XML.FixFloatSwapInstrument(spot, maturity, ytm, 6, 3, parrate, USD_ratecurve, USD_ratecurve)
        jacobienMatrix = pd.DataFrame(0, index = USD_ratecurve.df.index, columns = USD_ratecurve.df.index)
        
for i in USD_ratecurve.df.index:
    print("computation done at : ", datetime.now(tz=None))
    jacobienMatrix.loc[i] = USD_ratecurve.df.loc[i,'instrumentList'].getZCsensi()    
    USD_ratecurve.df.loc[i,'MV']  = USD_ratecurve.df.loc[i,'instrumentList'].getMV().iloc[0]

print(USD_ratecurve.df['MV']*1000000)

inv_J = pd.DataFrame(np.linalg.pinv(jacobienMatrix.values), jacobienMatrix.columns, jacobienMatrix.index)
# inv_J.dot(jacobienMatrix) cette quantité est presque une matrice diagonale
epsilon = -inv_J.dot(USD_ratecurve.df['MV'])
USD_ratecurve.df['ZCrate'] += epsilon
USD_ratecurve.cleanCache()

computation done at :  2020-04-20 21:22:14.939795
computation done at :  2020-04-20 21:22:15.060365
computation done at :  2020-04-20 21:22:15.148992
computation done at :  2020-04-20 21:22:15.220089
computation done at :  2020-04-20 21:22:15.285433
computation done at :  2020-04-20 21:22:15.347353
computation done at :  2020-04-20 21:22:15.612788
computation done at :  2020-04-20 21:22:15.969235
computation done at :  2020-04-20 21:22:16.426491
computation done at :  2020-04-20 21:22:17.001734
computation done at :  2020-04-20 21:22:17.762810
computation done at :  2020-04-20 21:22:18.592778
computation done at :  2020-04-20 21:22:19.504637
computation done at :  2020-04-20 21:22:20.516804
computation done at :  2020-04-20 21:22:21.637077
computation done at :  2020-04-20 21:22:22.992523
computation done at :  2020-04-20 21:22:24.692890
computation done at :  2020-04-20 21:22:27.021355
computation done at :  2020-04-20 21:22:29.910021
maturity
2018-02-12   -1.069023e-10
2018-03-12    

In [1485]:
spot = USD_ratecurve.spotdate

for index in USD_ratecurve.df.index:
    line = USD_ratecurve.df.loc[index]
    source = line['source']
    parrate = line['parrate']
    maturity = index
    if (source == 'deposit'):
        USD_ratecurve.df.loc[index, "instrumentList"] = XML.DepositInstrument(spot, maturity, parrate, USD_ratecurve)
    elif (source == 'swap'):
        ytm = line['YTM']
        USD_ratecurve.df.loc[index, "instrumentList"] = XML.FixFloatSwapInstrument(spot, maturity, ytm, 6, 3, parrate, USD_ratecurve, USD_ratecurve)
        jacobienMatrix = pd.DataFrame(0, index = USD_ratecurve.df.index, columns = USD_ratecurve.df.index)
        
for i in USD_ratecurve.df.index:
    print("computation done at : ", datetime.now(tz=None))
    jacobienMatrix.loc[i] = USD_ratecurve.df.loc[i,'instrumentList'].getZCsensi()    
    USD_ratecurve.df.loc[i,'MV']  = USD_ratecurve.df.loc[i,'instrumentList'].getMV().iloc[0]

print(USD_ratecurve.df['MV']*1000000)

inv_J = pd.DataFrame(np.linalg.pinv(jacobienMatrix.values), jacobienMatrix.columns, jacobienMatrix.index)
# inv_J.dot(jacobienMatrix) cette quantité est presque une matrice diagonale
epsilon = -inv_J.dot(USD_ratecurve.df['MV'])
USD_ratecurve.df['ZCrate'] += epsilon
USD_ratecurve.cleanCache()

computation done at :  2020-04-20 21:23:05.260733
computation done at :  2020-04-20 21:23:05.399177
computation done at :  2020-04-20 21:23:05.495668
computation done at :  2020-04-20 21:23:05.584468
computation done at :  2020-04-20 21:23:05.651544
computation done at :  2020-04-20 21:23:05.713661
computation done at :  2020-04-20 21:23:05.963977
computation done at :  2020-04-20 21:23:06.321857
computation done at :  2020-04-20 21:23:06.836778
computation done at :  2020-04-20 21:23:07.425869
computation done at :  2020-04-20 21:23:08.125758
computation done at :  2020-04-20 21:23:08.908662
computation done at :  2020-04-20 21:23:09.812058
computation done at :  2020-04-20 21:23:10.841523
computation done at :  2020-04-20 21:23:11.954943
computation done at :  2020-04-20 21:23:13.308821
computation done at :  2020-04-20 21:23:15.013575
computation done at :  2020-04-20 21:23:17.423360
computation done at :  2020-04-20 21:23:20.439422
maturity
2018-02-12    4.336809e-12
2018-03-12   -

In [1486]:
spot = USD_ratecurve.spotdate

for index in USD_ratecurve.df.index:
    line = USD_ratecurve.df.loc[index]
    source = line['source']
    parrate = line['parrate']
    maturity = index
    if (source == 'deposit'):
        USD_ratecurve.df.loc[index, "instrumentList"] = XML.DepositInstrument(spot, maturity, parrate, USD_ratecurve)
    elif (source == 'swap'):
        ytm = line['YTM']
        USD_ratecurve.df.loc[index, "instrumentList"] = XML.FixFloatSwapInstrument(spot, maturity, ytm, 6, 3, parrate, USD_ratecurve, USD_ratecurve)
        jacobienMatrix = pd.DataFrame(0, index = USD_ratecurve.df.index, columns = USD_ratecurve.df.index)
        
for i in USD_ratecurve.df.index:
    print("computation done at : ", datetime.now(tz=None))
    jacobienMatrix.loc[i] = USD_ratecurve.df.loc[i,'instrumentList'].getZCsensi()    
    USD_ratecurve.df.loc[i,'MV']  = USD_ratecurve.df.loc[i,'instrumentList'].getMV().iloc[0]

print(USD_ratecurve.df['MV']*1000000)

inv_J = pd.DataFrame(np.linalg.pinv(jacobienMatrix.values), jacobienMatrix.columns, jacobienMatrix.index)
# inv_J.dot(jacobienMatrix) cette quantité est presque une matrice diagonale
epsilon = -inv_J.dot(USD_ratecurve.df['MV'])
USD_ratecurve.df['ZCrate'] += epsilon
USD_ratecurve.cleanCache()

computation done at :  2020-04-20 21:23:36.650498
computation done at :  2020-04-20 21:23:36.771387
computation done at :  2020-04-20 21:23:36.861976
computation done at :  2020-04-20 21:23:36.939710
computation done at :  2020-04-20 21:23:37.019907
computation done at :  2020-04-20 21:23:37.092232
computation done at :  2020-04-20 21:23:37.359845
computation done at :  2020-04-20 21:23:37.711080
computation done at :  2020-04-20 21:23:38.174973
computation done at :  2020-04-20 21:23:38.756452
computation done at :  2020-04-20 21:23:39.433327
computation done at :  2020-04-20 21:23:40.214823
computation done at :  2020-04-20 21:23:41.111753
computation done at :  2020-04-20 21:23:42.118735
computation done at :  2020-04-20 21:23:43.285639
computation done at :  2020-04-20 21:23:44.748868
computation done at :  2020-04-20 21:23:46.485299
computation done at :  2020-04-20 21:23:48.795660
computation done at :  2020-04-20 21:23:51.685860
maturity
2018-02-12    4.336809e-12
2018-03-12    

In [1487]:
epsilon

maturity
2018-02-12    4.796773e-17
2018-03-12    2.387378e-16
2018-04-10   -4.572924e-17
2018-07-10    6.996399e-18
2019-01-10   -1.387779e-17
2020-01-10   -1.283854e-16
2021-01-10   -1.357757e-16
2022-01-10   -1.917082e-16
2023-01-10   -1.000043e-16
2024-01-10   -9.059171e-17
2025-01-10   -6.628141e-17
2026-01-10   -5.012595e-17
2027-01-10   -5.684419e-17
2028-01-10   -2.671199e-17
2030-01-10   -5.344760e-18
2033-01-10   -5.787521e-18
2038-01-10   -1.200348e-17
2043-01-10   -1.498845e-17
2048-01-10   -1.248156e-17
dtype: float64